In [1]:
# get your free API key at https://sec-api.io
api_key = "44ba705581dee21a56a223d5418b0d944702a85ac447047ed3a4b1f6f2ace0db"

In [ ]:
import requests
import json

filing_url = "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.html"
xbrl_converter_api_endpoint = "https://api.sec-api.io/xbrl-to-json"
api_key = "44ba705581dee21a56a223d5418b0d944702a85ac447047ed3a4b1f6f2ace0db"

final_url = xbrl_converter_api_endpoint + "?htm-url=" + filing_url + "&token=" + api_key
response = requests.get(final_url)
xbrl_json = json.loads(response.text)

# Save the full JSON to a file
with open("aapl_10k_xbrl.json", "w") as f:
    json.dump(xbrl_json, f, indent=2)

print("Saved full XBRL JSON to 'aapl_10q_xbrl.json'")

Saved full XBRL JSON to 'aapl_10q_xbrl.json'


In [17]:
import requests
import json
import pandas as pd

# 10-K filing URL of Apple
filing_url = "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.html"

# XBRL-to-JSON converter API endpoint
xbrl_converter_api_endpoint = "https://api.sec-api.io/xbrl-to-json"

# get your API key at https://sec-api.io
api_key = "44ba705581dee21a56a223d5418b0d944702a85ac447047ed3a4b1f6f2ace0db"

final_url = xbrl_converter_api_endpoint + "?htm-url=" + filing_url + "&token=" + api_key

# make request to the API
response = requests.get(final_url)

# load JSON into memory
xbrl_json = json.loads(response.text)

# income statement example
print(json.dumps(xbrl_json['StatementsOfIncome']['RevenueFromContractWithCustomerExcludingAssessedTax'][0:2], indent=1))
     


[
 {
  "decimals": "-6",
  "unitRef": "usd",
  "period": {
   "startDate": "2023-10-01",
   "endDate": "2024-09-28"
  },
  "segment": {
   "dimension": "srt:ProductOrServiceAxis",
   "value": "us-gaap:ProductMember"
  },
  "value": "294866000000"
 },
 {
  "decimals": "-6",
  "unitRef": "usd",
  "period": {
   "startDate": "2022-09-25",
   "endDate": "2023-09-30"
  },
  "segment": {
   "dimension": "srt:ProductOrServiceAxis",
   "value": "us-gaap:ProductMember"
  },
  "value": "298085000000"
 }
]


In [18]:
# convert XBRL-JSON of income statement to pandas dataframe
def get_income_statement(xbrl_json):
    income_statement_store = {}

    # iterate over each US GAAP item in the income statement
    for usGaapItem in xbrl_json['StatementsOfIncome']:
        values = []
        indicies = []

        for fact in xbrl_json['StatementsOfIncome'][usGaapItem]:
            # only consider items without segment. not required for our analysis.
            if 'segment' not in fact:
                index = fact['period']['startDate'] + '-' + fact['period']['endDate']
                # ensure no index duplicates are created
                if index not in indicies:
                    values.append(fact['value'])
                    indicies.append(index)                    

        income_statement_store[usGaapItem] = pd.Series(values, index=indicies) 

    income_statement = pd.DataFrame(income_statement_store)
    # switch columns and rows so that US GAAP items are rows and each column header represents a date range
    return income_statement.T 

income_statement = get_income_statement(xbrl_json)
     

In [19]:
income_statement

,2023-10-01-2024-09-28,2022-09-25-2023-09-30,2021-09-26-2022-09-24
RevenueFromContractWithCustomerExcludingAssessedTax,391035000000,383285000000,394328000000
CostOfGoodsAndServicesSold,210352000000,214137000000,223546000000
GrossProfit,180683000000,169148000000,170782000000
ResearchAndDevelopmentExpense,31370000000,29915000000,26251000000
SellingGeneralAndAdministrativeExpense,26097000000,24932000000,25094000000
OperatingExpenses,57467000000,54847000000,51345000000
OperatingIncomeLoss,123216000000,114301000000,119437000000
NonoperatingIncomeExpense,269000000,-565000000,-334000000
IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest,123485000000,113736000000,119103000000
IncomeTaxExpenseBenefit,29749000000,16741000000,19300000000


In [20]:
# convert XBRL-JSON of balance sheet to pandas dataframe
def get_balance_sheet(xbrl_json):
    balance_sheet_store = {}

    for usGaapItem in xbrl_json['BalanceSheets']:
        values = []
        indicies = []

        for fact in xbrl_json['BalanceSheets'][usGaapItem]:
            # only consider items without segment.
            if 'segment' not in fact:
                index = fact['period']['instant']

                # avoid duplicate indicies with same values
                if index in indicies:
                    continue
                    
                # add 0 if value is nil
                if "value" not in fact:
                    values.append(0)
                else:
                    values.append(fact['value'])

                indicies.append(index)                    

            balance_sheet_store[usGaapItem] = pd.Series(values, index=indicies) 

    balance_sheet = pd.DataFrame(balance_sheet_store)
    # switch columns and rows so that US GAAP items are rows and each column header represents a date instant
    return balance_sheet.T

balance_sheet = get_balance_sheet(xbrl_json)

In [21]:
balance_sheet

,2021-09-25,2022-09-24,2023-09-30,2024-09-28
CashAndCashEquivalentsAtCarryingValue,NaN,NaN,29965000000,29943000000
MarketableSecuritiesCurrent,NaN,NaN,31590000000,35228000000
AccountsReceivableNetCurrent,NaN,NaN,29508000000,33410000000
NontradeReceivablesCurrent,NaN,NaN,31477000000,32833000000
InventoryNet,NaN,NaN,6331000000,7286000000
OtherAssetsCurrent,NaN,NaN,14695000000,14287000000
AssetsCurrent,NaN,NaN,143566000000,152987000000
MarketableSecuritiesNoncurrent,NaN,NaN,100544000000,91479000000
PropertyPlantAndEquipmentNet,NaN,NaN,43715000000,45680000000
OtherAssetsNoncurrent,NaN,NaN,64758000000,74834000000


In [22]:
def get_cash_flow_statement(xbrl_json):
    cash_flows_store = {}

    for usGaapItem in xbrl_json['StatementsOfCashFlows']:
        values = []
        indicies = []

        for fact in xbrl_json['StatementsOfCashFlows'][usGaapItem]:        
            # only consider items without segment.
            if 'segment' not in fact:
                # check if date instant or date range is present
                if "instant" in fact['period']:
                    index = fact['period']['instant']
                else:
                    index = fact['period']['startDate'] + '-' + fact['period']['endDate']

                # avoid duplicate indicies with same values
                if index in indicies:
                    continue

                if "value" not in fact:
                    values.append(0)
                else:
                    values.append(fact['value'])

                indicies.append(index)                    

        cash_flows_store[usGaapItem] = pd.Series(values, index=indicies) 


    cash_flows = pd.DataFrame(cash_flows_store)
    return cash_flows.T
    
cash_flows = get_cash_flow_statement(xbrl_json)

In [23]:
cash_flows

,2021-09-25,2021-09-26-2022-09-24,2022-09-24,2022-09-25-2023-09-30,2023-09-30,2023-10-01-2024-09-28,2024-09-28
CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents,35929000000,NaN,24977000000,NaN,30737000000,NaN,29943000000
NetIncomeLoss,NaN,99803000000,NaN,96995000000,NaN,93736000000,NaN
DepreciationDepletionAndAmortization,NaN,11104000000,NaN,11519000000,NaN,11445000000,NaN
ShareBasedCompensation,NaN,9038000000,NaN,10833000000,NaN,11688000000,NaN
OtherNoncashIncomeExpense,NaN,-1006000000,NaN,2227000000,NaN,2266000000,NaN
IncreaseDecreaseInAccountsReceivable,NaN,1823000000,NaN,1688000000,NaN,3788000000,NaN
IncreaseDecreaseInOtherReceivables,NaN,7520000000,NaN,-1271000000,NaN,1356000000,NaN
IncreaseDecreaseInInventories,NaN,-1484000000,NaN,1618000000,NaN,1046000000,NaN
IncreaseDecreaseInOtherOperatingAssets,NaN,6499000000,NaN,5684000000,NaN,11731000000,NaN
IncreaseDecreaseInAccountsPayable,NaN,9448000000,NaN,-1889000000,NaN,6020000000,NaN


In [24]:
def get_cash_flow_statement(xbrl_json):
    cash_flows_store = {}

    for usGaapItem in xbrl_json['StatementsOfCashFlows']:
        values = []
        indicies = []

        for fact in xbrl_json['StatementsOfCashFlows'][usGaapItem]:        
            # only consider items without segment.
            if 'segment' not in fact:
                # For instant dates (like cash balances)
                if "instant" in fact['period']:
                    date_str = fact['period']['instant']
                    # Check if it's a September year-end date (annual)
                    if '-09-' in date_str:
                        index = date_str
                
                # For period ranges (like income items)
                elif "startDate" in fact['period']:
                    start = fact['period']['startDate']
                    end = fact['period']['endDate']
                    # Check if it's a full fiscal year range (Sept-Sept)
                    if '-09-' in start and '-09-' in end:
                        index = start + '-' + end
                else:
                    continue  # Skip if not annual period

                # Only add if we found an annual period
                if 'index' in locals():
                    # avoid duplicate indicies with same values
                    if index in indicies:
                        continue

                    if "value" not in fact:
                        values.append(0)
                    else:
                        values.append(fact['value'])

                    indicies.append(index)                    

        if indicies:  # Only add if we found annual data
            cash_flows_store[usGaapItem] = pd.Series(values, index=indicies) 

    cash_flows = pd.DataFrame(cash_flows_store)
    return cash_flows.T
    
cash_flows = get_cash_flow_statement(xbrl_json)

In [25]:
cash_flows  # This will display the dataframe in Jupyter

,2021-09-25,2021-09-26-2022-09-24,2022-09-24,2022-09-25-2023-09-30,2023-09-30,2024-09-28
CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents,35929000000,NaN,24977000000,NaN,30737000000,29943000000
NetIncomeLoss,NaN,99803000000,NaN,96995000000,NaN,93736000000
DepreciationDepletionAndAmortization,NaN,11445000000,NaN,11519000000,NaN,NaN
ShareBasedCompensation,NaN,11688000000,NaN,10833000000,NaN,NaN
OtherNoncashIncomeExpense,NaN,2266000000,NaN,2227000000,NaN,NaN
IncreaseDecreaseInAccountsReceivable,NaN,3788000000,NaN,1688000000,NaN,NaN
IncreaseDecreaseInOtherReceivables,NaN,1356000000,NaN,-1271000000,NaN,NaN
IncreaseDecreaseInInventories,NaN,1046000000,NaN,1618000000,NaN,NaN
IncreaseDecreaseInOtherOperatingAssets,NaN,11731000000,NaN,5684000000,NaN,NaN
IncreaseDecreaseInAccountsPayable,NaN,6020000000,NaN,-1889000000,NaN,NaN


In [2]:
import pandas as pd
from sqlalchemy import create_engine

def get_db_engine(user, password, host, port, db_name):
    """
    Creates and returns a SQLAlchemy engine for connecting to a PostgreSQL database.
    """
    DATABASE_URL = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{db_name}"
    return create_engine(DATABASE_URL)


In [3]:
def save_to_postgres(engine, df, table_name):
    """
    Saves a Pandas DataFrame to PostgreSQL.
    """
    try:
        df.to_sql(table_name, engine, if_exists='replace', index=True)
        print(f"✅ Table '{table_name}' successfully saved to the database.")
    except Exception as e:
        print(f"❌ Error saving table '{table_name}': {e}")


In [4]:
def upload_financial_data(balance_sheet, cash_flows, income_statement):
    # Establish connection
    engine = get_db_engine(
        user='aman',
        password='Aman2003',
        host='localhost',
        port=5432,
        db_name='financial_data'
    )
    
    # Upload tables
    save_to_postgres(engine, balance_sheet, 'balance_sheet')
    save_to_postgres(engine, cash_flows, 'cash_flow')
    save_to_postgres(engine, income_statement, 'income_statement')
    
    # Close the connection
    engine.dispose()
    print("✅ All tables uploaded successfully!")


In [5]:
upload_financial_data(balance_sheet, cash_flows, income_statement)


NameError: name 'balance_sheet' is not defined

In [6]:
# To verify if tables are properly saved in PostgreSQL
engine = get_db_engine('aman', 'Aman2003', 'localhost', 5432, 'financial_data')

# Check each table
for table_name in ['balance_sheet', 'cash_flow', 'income_statement']:
    df = pd.read_sql(f"SELECT * FROM {table_name};", engine)
    print(f"\n🔍 Preview of '{table_name}' table:")
    print(df.head())
    
engine.dispose()



🔍 Preview of 'balance_sheet' table:
                                   index 2021-09-25 2022-09-24   2023-09-30  \
0  CashAndCashEquivalentsAtCarryingValue       None       None  29965000000   
1            MarketableSecuritiesCurrent       None       None  31590000000   
2           AccountsReceivableNetCurrent       None       None  29508000000   
3             NontradeReceivablesCurrent       None       None  31477000000   
4                           InventoryNet       None       None   6331000000   

    2024-09-28  
0  29943000000  
1  35228000000  
2  33410000000  
3  32833000000  
4   7286000000  

🔍 Preview of 'cash_flow' table:
                                               index   2021-09-25  \
0  CashCashEquivalentsRestrictedCashAndRestricted...  35929000000   
1                                      NetIncomeLoss         None   
2               DepreciationDepletionAndAmortization         None   
3                             ShareBasedCompensation         None   
4       